In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("imdb.csv", low_memory=False)

In [3]:
data = df.drop(columns = ['cast'])

In [4]:
data=data.drop(data[data['stars'].isnull() &  data['writer'].isnull() & data['director'].isnull()].index).reset_index(drop=True)


In [5]:
data = data.dropna(subset=['averageRating']).reset_index(drop=True)

In [6]:
data.sort_values('id', inplace=True)

In [7]:
data.drop_duplicates(keep="first",inplace=True) 

In [8]:
data = data.reset_index(drop=True)

In [9]:
def caculateScore(columnName):
    names = data[['averageRating', columnName]]
    my_dict = {}
    n = len(names)
    for i in range(n):
        avgR, name = names.iloc[i,:]
        if type(name) == str:
            avgR = float(avgR)
            name = name.split(',')
            for s in name:
                if s not in my_dict.keys():
                    my_dict[s] = [avgR, 1]
                else:
                    a = my_dict[s]
                    a = [avgR + a[0], a[1] + 1]
                    my_dict[s] = a
#     v_max = 0.0
    for a in my_dict:
        score, n = my_dict[a]
        my_dict[a] = score/n
#         if v_max < my_dict[a]:
#             v_max = my_dict[a]
    n = len(data)
    data['score_'+columnName] =  pd.Series(np.zeros(n), index=data.index)
    score_name = 'score_'+columnName
    for i in range(n):
        name = data.at[i,columnName]
        if type(name) == str:
            name = name.split(',')
            for s in name:
                data.at[i,score_name] = data.at[i,score_name] + my_dict[s]
            data.at[i,score_name] = data.at[i,score_name] 

In [10]:
caculateScore('stars')

In [11]:
caculateScore('director')

In [12]:
caculateScore('writer')

In [13]:
caculateScore('country')

In [14]:
data=data.drop(columns = ['stars', 'writer', 'director', 'country', 'id'])

In [15]:
data['averageRating'] = data['averageRating'].round()

In [16]:
currency_unit = {"$"  : 1,
                 "FRF": 0.1846,
                 "SEK": 0.119,
                 "DEM": 0.6193,
                 "GBP": 1.335,
                 "ESP": 0.007279,
                 "ITL": 0.0006255,
                 "INR": 0.01354,
                 "JPY": 0.009611,
                 "RUR": 0.01356,
                 "AUD": 0.7426,
                 "CAD": 0.7816,
                 "HKD": 0.129,
                 "EUR": 1.211 ,
                 "DKK": 0.1627,
                 "NOK": 0.1144,
                 "ZAR": 0.06597,
                 "BRL": 0.1961,
                 "THB": 0.0332,
                 "CNY": 0.153,
                 "HUF": 0.003362,
                 "KRW": 0.0009222,
                 "ARS": 0.01227,
                 "NZD": 0.7033,
                 "CZK": 0.04571,
                 "IEP": 1.538,
                 "SGD": 0.7479,
                 "ILS": 0.3072,
                 "BGL": 0.0006192,
                 "ATS": 0.08802,
                 "BND": 0.7473, 
                 "CHF": 1.122,
                 "VEB": 0.0000000004,
                 "PLN": 0.06605,
                 "CLP": 0.001342,
                 "PKR": 0.006242,
                 "NGN": 0.002607,
                 "COP": 0.0002876,
                 "MXN": 0.05033,
                 "TRL": 0.0000000128,
                 "IDR": 0.00007059
                }

In [17]:
def convert_monney(name):
    for i in range(len(data)):
        x = data.at[i,name]
        if type(x) == str:
            if x[0] == '$':
                x = x.replace('$','')
                x = x.replace(',','')
                x = int(x)
                data.at[i,name] = x
            else:
                a = x[0:3]
                x = x[3:]
                x = x.replace(',','')
                x = float(x)
                x = round(x * currency_unit[a])
                data.at[i,name] = x

    m = round(data[name].mean() ) 
    data[name] = data[name].fillna(m)

In [18]:
convert_monney('budget')

In [19]:
convert_monney('cumulative')

In [20]:
data

,averageRating,budget,cumulative,storyLine,score_stars,score_director,score_writer,score_country
0,5.0,20597339,43925770,Panoramic view of the 1901 Pan-American Ex...,0.000000,5.400000,0.000000,12.923191
1,8.0,5538,43925770,An association of astronomers has convened...,0.000000,8.200000,24.600000,13.514021
2,5.0,20597339,43925770,"The Rev. Mr. Hollister, nearing the last m...",15.600000,6.450000,5.200000,12.923191
3,8.0,385907,43925770,Intolerance and its terrible effects are e...,23.250000,6.450000,7.700000,12.923191
4,7.0,20597339,43925770,Paul Korner is a homosexual musician who f...,21.257143,7.000000,7.000000,19.806525
...,...,...,...,...,...,...,...,...
19716,7.0,20597339,30996615,An obsessive CEO of a company meets a ragg...,20.740000,6.700000,12.333333,6.484793
19717,8.0,20597339,43925770,A seven-part docuseries about the unsolved...,0.000000,31.668421,0.000000,6.180334
19718,6.0,20597339,43925770,"In 2010 David Crowley, an Iraq veteran, as...",0.000000,6.533333,18.600000,6.180334
19719,6.0,20597339,43925770,"In July 2015, Ashley Madison, 'The Origina...",0.000000,11.000000,16.500000,6.180334
